In [ ]:
from typing import Dict, Tuple

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

%matplotlib Inline
%config InlineBackend.figure_format = "retina"

## Defining Generating Functions
---

In [ ]:
def E(q: float, r0: Tuple[float, float], x: np.ndarray, y: np.ndarray) -> Tuple[float, float]:
    """
    Return the electric field vector E = (Ex, Ey) due to a charge q at position r0.

    Args:
        q (float): the charge value.
        r0 (Tuple[float, float]): position of the charge.
        x (np.ndarray): x positions at which to calculate the field.
        y (np.ndarray): y positions at which to calculate the field.

    Returns:
        The field vector (Ex, Ey) as a tuple.
    """
    denominator = np.hypot(x - r0[0], y - r0[1]) ** 3
    return q * (x - r0[0]) / denominator, q * (y - r0[1]) / denominator

In [ ]:
def generate_multipole(order: int, skew: bool = False) -> Dict[Tuple[float, float], float]:
    """
    Create a multipole with 2^order charges of alternating sign, equally spaced on the unit circle.

    Args:
        order (int): the order of the multipole.
        skew (bool): if True, the multipole will be a skew one. Defaults to False.

    Returns:
        A dictionary of {position: charge_value} for each charge in the multipole.
    """
    n_charges = 2 ** int(order)
    offset = np.pi / 4 if skew else 0
    charges = {}

    for i in range(n_charges):
        q = i % 2 * 2 - 1  # to alternate the signs
        charges[(np.cos(offset + 2 * np.pi * i / n_charges), np.sin(offset + 2 * np.pi * i / n_charges))] = q
    return charges

In [ ]:
def calculate_field_map(n_x: int, n_y: int, charges: Dict[Tuple[float, float], float]) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculates the electric field vector values for all points in the grid of points.

    Args:
        n_x (int): number of points on the x axis.
        n_y (int): number of points on the y axis.
        charges (Dict[Tuple[float, float], float]): dict of charges positions and charge value, as returned by 'generate_multipole'.

    Returns:
        A tuple of the Ex and Ey arrays.
    """
    Ex, Ey = np.zeros((n_y, n_x)), np.zeros((n_y, n_x))

    for charge_position, charge in charges.items():
        ex, ey = E(q=charge, r0=charge_position, x=X, y=Y)
        Ex += ex
        Ey += ey
    return Ex, Ey

## Choosing and Order & Creating Grid
---

In [ ]:
# Choose the multipole order here
order = 2

In [ ]:
# Create a grid of x, y points
n_x, n_y = 128, 128

# Modify limits here depending on the order for better visual (remember the charges are on the unit circle)
x = np.linspace(-1.5, 1.5, n_x)
y = np.linspace(-1.5, 1.5, n_y)

X, Y = np.meshgrid(x, y)

## Plotting Field Lines
---

### Normal Multipole

In [ ]:
charges = generate_multipole(order)
Ex, Ey = calculate_field_map(n_x, n_y, charges)

In [ ]:
fig = plt.figure(figsize=(20, 12.5))
axis = fig.add_subplot(111)

# Plot the streamlines with an appropriate colormap and arrow style
color = 2 * np.log(np.hypot(Ex, Ey))
axis.streamplot(x, y, Ex, Ey, color=color, linewidth=1, cmap=plt.cm.inferno, density=2, arrowstyle="-|>", arrowsize=1.5)

# Add filled circles for the charges themselves
charge_colors = {True: "#aa0000", False: "#0000aa"}
for charge_position, charge in charges.items():
    axis.add_artist(Circle(charge_position, 0.05, color=charge_colors[charge > 0]))

axis.set_aspect("equal")
axis.set_axis_off()

# plt.savefig(f"multipole_order_{order}", dpi=1000)

### Skew Multipole

In [ ]:
skew_charges = generate_multipole(order, skew=True)
Ex_skew, Ey_skew = calculate_field_map(n_x, n_y, skew_charges)

In [ ]:
fig = plt.figure(figsize=(20, 12.5))
axis = fig.add_subplot(111)

# Plot the streamlines with an appropriate colormap and arrow style
color = 2 * np.log(np.hypot(Ex_skew, Ey_skew))
axis.streamplot(x, y, Ex_skew, Ey_skew, color=color, linewidth=1, cmap=plt.cm.inferno, density=2, arrowstyle="-|>", arrowsize=1.5)

# Add filled circles for the charges themselves
charge_colors = {True: "#aa0000", False: "#0000aa"}
for charge_position, charge in skew_charges.items():
    axis.add_artist(Circle(charge_position, 0.05, color=charge_colors[charge > 0]))

axis.set_aspect("equal")
axis.set_axis_off()

# plt.savefig(f"skew_multipole_order_{order}", dpi=1000)

---